In [ ]:
import json

file_path = "memory.json"
with open(file_path, "r") as d:
  memory = json.loads(d.read())

concept = "Quasi-Connectivity and Quantum Computing"
prompt = f"""
You are a helpful assisstant, specialising in the area of {concept}. 
Give a short definition and explanation on the basics of the subject in 100 words.  
Here is a record of previous prompts and responses: {memory}.
Do not reiterate information present. Build upon it where possible. 
Make explicit references to content in memory when possible.
Include intuitive line breaks in response for formatting purposes.
""" 
message = f"Explain {concept} to me"

prompt2 = f"""
You are a waether assisstant. Run thi
"""

In [68]:
print(memory)

[{'user': 'Explain Quantum Physics to me.', 'AI_response': 'Hello there! As a helpful assistant specializing in Quantum Physics, I\'d be delighted to give you a basic rundown.\n\n---\n\n### What is Quantum Physics?\n\n**Short Definition:** Quantum Physics is the fundamental theory describing the nature and behavior of matter and energy at the **atomic and subatomic levels**. It\'s a radical departure from classical physics, which effectively describes the world at larger scales.\n\n---\n\n### The Basics Explained:\n\nImagine you\'re trying to understand how a watch works, but all you have is a magnifying glass that lets you see gears, springs, and tiny components. Classical physics is like looking at the entire watch and understanding its overall function. Quantum physics is like zooming in so far that you realize the tiny components themselves behave in incredibly strange and counter-intuitive ways.\n\nHere are the core concepts:\n\n1.  **Quantization of Energy (and other properties):

In [66]:
from google import genai
import os
import json

client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt,
)

print(response.text)
format = {
  "user": message,
  "AI_response": response.text
}

with open(file_path, "r") as file:
  data = json.loads(file.read())

data.append(format)
with open(file_path, "w") as f:
  f.write(json.dumps(data, indent=4))


Quasi-Connectivity, as previously discussed, describes a network state that is "almost" fully connected but inherently vulnerable. It signifies reliance on critical links, meaning removal of key elements can easily fragment the system, highlighting areas of fragility.

Quantum Computing, building on our understanding of Quantum Physics, leverages phenomena like **superposition** (where quantum bits, or qubits, exist in multiple states simultaneously) and entanglement. Unlike classical bits, qubits allow for vastly parallel processing. This computational power enables solutions to problems intractable for conventional computers, revolutionizing fields from material science to cryptography by exploiting nature's fundamental quantum properties.


In [ ]:
import requests
import os

def get_open_weather():
  API_KEY = os.getenv("OPENWEATHER_API_KEY")
  CITY_NAME = "Amsterdam"
  BASE_URL = "https://api.openweathermap.org/data/2.5/weather"

  params = {
      "q": CITY_NAME,
      "appid": API_KEY,
      "units": "metric" 
  }

  try:
      response = requests.get(BASE_URL, params=params)
      response.raise_for_status()
      data = response.json()

      weather = {
          "city": data["name"],
          "temperature": data["main"]["temp"],
          "description": data["weather"][0]["description"],
          "humidity": data["main"]["humidity"],
          "wind_speed": data["wind"]["speed"]
      }

      print(f"Weather in {weather['city']}:")
      print(f"- Temperature: {weather['temperature']}°C")
      print(f"- Description: {weather['description'].capitalize()}")
      print(f"- Humidity: {weather['humidity']}%")
      print(f"- Wind Speed: {weather['wind_speed']} m/s")

  except requests.exceptions.RequestException as e:
      print(f"Error fetching weather data: {e}")

get_open_weather()


Weather in Amsterdam:
- Temperature: 18.42°C
- Description: Light rain
- Humidity: 89%
- Wind Speed: 4.83 m/s


In [ ]:
import os
import pandas as pd
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer


api_key = os.environ.get("PINECONE_API_KEY")
if not api_key:
    raise ValueError("PINECONE_API_KEY environment variable not set")

pc = Pinecone(api_key=api_key)

model = SentenceTransformer('all-MiniLM-L6-v2')
embedding_dim = 384
index_name = "genai-quickstart"

if index_name not in pc.list_indexes().names():
    print(f"Creating a new index: {index_name}")
    pc.create_index(
        name=index_name,
        dimension=embedding_dim,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )
    print("Index created successfully.")
else:
    print(f"Index '{index_name}' already exists.")

index = pc.Index(index_name)
print("\nConnected to index. Index stats:")
print(index.describe_index_stats())


data = {
    'id': ['doc1', 'doc2', 'doc3', 'doc4'],
    'text': [
        "The cat sat on the mat.",
        "A dog played in the sunny park.",
        "Artificial intelligence will transform the world.",
        "Generative AI models can create text and images."
    ],
    'category': ['animals', 'animals', 'technology', 'technology']
}
df = pd.DataFrame(data)

print("\n--- Preparing and Embedding Data ---")
df['embedding'] = df['text'].apply(lambda x: model.encode(x).tolist())

print("Data embedded. Now upserting to Pinecone...")

vectors_to_upsert = []
for i, row in df.iterrows():
    vector = {
        "id": row['id'],
        "values": row['embedding'],
        "metadata": {"text": row['text'], "category": row['category']}
    }
    vectors_to_upsert.append(vector)

index.upsert(vectors=vectors_to_upsert)

print("Upsert complete. Index stats after upsert:")
print(index.describe_index_stats())

print("\n--- Querying the Index ---")
query_text = "What is the future of AI?"

query_embedding = model.encode(query_text).tolist()

results = index.query(
    vector=query_embedding,
    top_k=2,  
    include_metadata=True
)

print(f"Query: '{query_text}'")
print("\nTop results from Pinecone:")
for match in results['matches']:
    print(f"  - ID: {match['id']}")
    print(f"    Score: {match['score']:.4f}")
    print(f"    Text: {match['metadata']['text']}")
    print(f"    Category: {match['metadata']['category']}\n")

Creating a new index: genai-quickstart
Index created successfully.

Connected to index. Index stats:
{'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

--- Preparing and Embedding Data ---
Data embedded. Now upserting to Pinecone...


c:\Users\KARANJA\OneDrive\Desktop\DirectEd_Module_3\.venv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Upsert complete. Index stats after upsert:
{'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

--- Querying the Index ---


c:\Users\KARANJA\OneDrive\Desktop\DirectEd_Module_3\.venv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Query: 'What is the future of AI?'

Top results from Pinecone:
